# **Approche supervisée**

In [1]:
# Pour manipuler nos données
import pandas as pd


import numpy as np  # pour faire les np.zeros

# Pour les vectorisations de type bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Gensim - Pour la vectorisation de type Word2Vec
from gensim.models import Word2Vec

# Utilisé pour l'embedding avec Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model

# Visualisations
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Mesures de performance des prédictions
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer  # Utilisé dans la fonction de qualité des prédictions

# Mesures de durée d'éxécution
import time

# Utilisé lors des sauvagardes et chargement de données/objets
import pickle
import os  # Aussi utilisé pour connaitre le nombre de CPU

# Éviter les Warnings
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)

Ce notebook fonctionne avec : 

- Python 3.10.9
- Anaconda 23.3.1
- Wordcloud 1.9.2
- Gensim 4.3.0


- Mesure de la durée d'exécution du notebook : 

In [2]:
t_notebook = time.time()

## **Importations des données**

In [3]:
with open('data_clean.pkl', 'rb') as fichier:
    data = pickle.load(fichier)

data

,date,corpus,corpus_dl,tags
0,2022-05-08 21:22:05,"[firebase, testing, jest, error, assertion, cl...","[firebase, testing, with, jest, throws, error,...","[firebase, jestjs, reactjs]"
1,2022-05-08 21:49:08,"[block, hack, language, benefit, block, job, w...","[concurrent, block, in, hacklang, since, hack,...","[async-await, concurrency]"
2,2022-05-08 21:49:57,"[type, function, typescript, function, type, i...","[can, you, set, a, type, for, a, function, in,...",[typescript]
3,2022-05-08 21:51:00,"[store, service, account, looking, expo, appli...","[expo, eas, submit, where, to, store, service,...",[expo]
4,2022-05-08 22:32:53,"[store, retrieve, structure, type, c, copying,...","[reliably, and, portably, store, and, retrieve...",[c]
...,...,...,...,...
46493,2022-03-21 08:52:43,"[option, option, know, option, case, option, o...","[how, to, know, if, a, select2, has, options, ...","[javascript, jquery]"
46494,2022-03-21 08:54:03,"[ring, plot, attempt, gap, use, plot, paint, r...","[matplotlib, how, to, plot, a, closed, ring, i...","[matplotlib, python]"
46495,2022-03-21 09:02:54,"[security, problem, terraform, dependency, loc...","[which, security, problem, does, terraform, ch...",[terraform]
46496,2022-03-21 13:54:21,"[studio, creation, blob, container, error, mes...","[visual, studio, 2022, with, azurite, integrat...",[azure]


## **Option d'échantillonages pour la réalisation de tests**

In [4]:
echantillon = False
taille_echantillon = 5000

if echantillon:
    data = data.sample(taille_echantillon, random_state=42)  # random_state car en fonction de l'échantillonage
                                                             # les résultats peuvent changer et pour pouvoir
                                                             # comparer nos tunings d'hyperparamètres, nous devons
                                                             # avoir des résultats reproductibles

data

,date,corpus,corpus_dl,tags
0,2022-05-08 21:22:05,"[firebase, testing, jest, error, assertion, cl...","[firebase, testing, with, jest, throws, error,...","[firebase, jestjs, reactjs]"
1,2022-05-08 21:49:08,"[block, hack, language, benefit, block, job, w...","[concurrent, block, in, hacklang, since, hack,...","[async-await, concurrency]"
2,2022-05-08 21:49:57,"[type, function, typescript, function, type, i...","[can, you, set, a, type, for, a, function, in,...",[typescript]
3,2022-05-08 21:51:00,"[store, service, account, looking, expo, appli...","[expo, eas, submit, where, to, store, service,...",[expo]
4,2022-05-08 22:32:53,"[store, retrieve, structure, type, c, copying,...","[reliably, and, portably, store, and, retrieve...",[c]
...,...,...,...,...
46493,2022-03-21 08:52:43,"[option, option, know, option, case, option, o...","[how, to, know, if, a, select2, has, options, ...","[javascript, jquery]"
46494,2022-03-21 08:54:03,"[ring, plot, attempt, gap, use, plot, paint, r...","[matplotlib, how, to, plot, a, closed, ring, i...","[matplotlib, python]"
46495,2022-03-21 09:02:54,"[security, problem, terraform, dependency, loc...","[which, security, problem, does, terraform, ch...",[terraform]
46496,2022-03-21 13:54:21,"[studio, creation, blob, container, error, mes...","[visual, studio, 2022, with, azurite, integrat...",[azure]


## **Vectorisations du corpus**

### **Vectorisations de type *bag-of-words* classique**

Nous utilisons *data['corpus']*, le corpus avec traitement complet (suppression des stopwords, des mots rares, etc).

In [5]:
# Chaque document de data['corpus'] est sous la forme d'une liste de mot.
# CountVectorizer() prend en entrée un string. Nous allons donc concaténer chaque document.
preprocessed_corpus = [' '.join(doc) for doc in data['corpus']]

# max_features : ne retenir que les x mots les plus fréquents
# Sans cette limite, le nombre de features serait égal
# au nombre de mots dans le vocabulaire
max_features = 500

# Création de CountVectorizer et vectorisation des données textuelles
vectorizer_count = CountVectorizer(max_features=max_features)
vectorized_corpus_count = vectorizer_count.fit_transform(preprocessed_corpus)

# Afficher les caractéristiques de la matrice vectorisée
print(f"Nombre d'observations (documents) : {vectorized_corpus_count.shape[0]}")
print(f"Nombre de variables (features) : {vectorized_corpus_count.shape[1]}")
print("Premier document vectorisé (100 premières features) :")
print(vectorized_corpus_count.toarray()[0][:100])

Nombre d'observations (documents) : 46498
Nombre de variables (features) : 500
Premier document vectorisé (100 premières features) :
[0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]


### **Vectorisations de type *bag-of-words* TF-IDF**

Nous utilisons encore *data['corpus']*, le corpus avec traitement complet (suppression des stopwords, des mots rares, etc).

In [6]:
# max_features : ne retenir que les x mots les plus fréquents
# Sans cette limite, le nombre de features serait égal
# au nombre de mots dans le vocabulaire
max_features = 500

# Création de TfidfVectorizer et vectorisation des données textuelles
vectorizer_tfidf = TfidfVectorizer(max_features=max_features)
vectorized_corpus_ftidf = vectorizer_tfidf.fit_transform(preprocessed_corpus)  # On réutilise preprocessed_corpus crée lors
                                                                               # de la vectorisation BoW classique

# Afficher les caractéristiques de la matrice vectorisée
print(f"Nombre d'observations (documents) : {vectorized_corpus_ftidf.shape[0]}")
print(f"Nombre de variables (features) : {vectorized_corpus_ftidf.shape[1]}")
print("Premier document vectorisé (100 premières features) :")
print(vectorized_corpus_ftidf.toarray()[0][:100])

Nombre d'observations (documents) : 46498
Nombre de variables (features) : 500
Premier document vectorisé (100 premières features) :
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.20109768
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.16017539 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.09734    0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.11521863
 0.         0.         0.

### **Vectorisations de type *Word/Sentence Embedding***

#### **A). Word2Vec**

Nous utilisons de nouveau *data['corpus']*, le corpus avec traitement complet (suppression des stopwords, des mots rares, etc).

Avant de démarrer, regardons les tailles des documents de notre corpus avec traitement complet (data['corpus']), ceci va nous servir par la suite : 

In [7]:
min_length = min(len(document) for document in data['corpus'])

# Afficher la taille de la plus grande liste
print(f"La taille du plus court document est : {min_length}")

max_length = max(len(document) for document in data['corpus'])

# Afficher la taille de la plus grande liste
print(f"La taille du plus long document est : {max_length}")

# Calculer la taille moyenne
average_length = sum(len(document) for document in data['corpus']) / len(data['corpus'])

# Afficher la taille moyenne
print(f"La taille moyenne des documents est : {average_length:.1f}")

La taille du plus court document est : 1
La taille du plus long document est : 846
La taille moyenne des documents est : 26.3


- Choix entre CBOW et Skip-gram à utiliser dans Word2Vec.

**CBOW (Continuous Bag of Words) :** vise à prédire un mot étant donné son contexte, c'est-à-dire étant donné les mots qui en sont proches dans le texte. S'entraîne plus rapidement que Skip-Gram (mieux adapté pour des dataset volumineux) et peut mieux représenter les mots plus fréquents.

**Skip-gram :** architecture symétrique visant à prédire les mots du contexte étant donné un mot en entrée. Fonctionne bien avec de petits datsets et peut mieux représenter les mots moins fréquents.

Dans notre cas, notamment en raison de la taille du dataset, nous utiliserons **CBOW**.

- Création et entraînement du modèle Word2Vec

In [8]:
# Paramètres à passer dans le modèle : 

w2v_size = 100  # Taille max des vecteurs
w2v_window = 5  # Taille du contexte
w2v_min_count = 1  # Minimum d'occurences d'un mot pour être pris en compte
w2v_epochs = 50  # Nombre de passes sur tout le corpus

documents = data['corpus']

# Création et entraînement du modèle Word2Vec

print("Création et entraînement du modèle Word2Vec...")
w2v_model = Word2Vec(min_count=w2v_min_count,
                     window=w2v_window,
                     vector_size=w2v_size,
                     sg=0,  # 0 : CBOX, 1 : Skipgram
                     seed=42,
                     workers=(os.cpu_count() - 1)  # Utiliser total des cpu - 1
)

w2v_model.build_vocab(documents)

w2v_model.train(documents,
                total_examples=w2v_model.corpus_count,
                epochs=w2v_epochs
)

model_vectors = w2v_model.wv

w2v_words = model_vectors.index_to_key

print("Taille du vocabulaire : %i" % len(w2v_words))
print("Word2Vec entraîné")

Création et entraînement du modèle Word2Vec...
Taille du vocabulaire : 3707
Word2Vec entraîné


- Test du modèle entraîné pour vérifier qu'il a réussi à capter les similarités entre les mots

In [9]:
# Mots similaires à un mot donné
mot_a_tester = "python"
print(f"Mots similaires à '{mot_a_tester}' :\n")
similar_words = w2v_model.wv.most_similar(positive=[mot_a_tester])
for word, similarity in similar_words:
    print(f"{word} : {similarity:.4f}")

print("\n--------------------\n")

# Similarité entre deux mots
mot_a_tester_1 = "python"
mot_a_tester_2 = "virtualenv"
similarite = w2v_model.wv.similarity(mot_a_tester_1, mot_a_tester_2)
print(f"Similarité entre '{mot_a_tester_1}' et '{mot_a_tester_2}' : {similarite:.4f}")

print("\n--------------------\n")

# Trouver l'intrus
mot_intrus_1 = "python"
mot_intrus_2 = "javascript"
mot_intrus_3 = "image"
intrus = w2v_model.wv.doesnt_match([mot_intrus_1, mot_intrus_2, mot_intrus_3])
print(f"L'intrus parmi ces mots ['{mot_intrus_1}', '{mot_intrus_2}', '{mot_intrus_3}'] est : '{intrus}'.")

Mots similaires à 'python' :

pip : 0.4794
modulenotfounderror : 0.4427
conda : 0.4255
poetry : 0.4159
venv : 0.4156
pypi : 0.4057
setuptool : 0.4013
pyenv : 0.4010
interpreter : 0.3994
gdal : 0.3952

--------------------

Similarité entre 'python' et 'virtualenv' : 0.3547

--------------------

L'intrus parmi ces mots ['python', 'javascript', 'image'] est : 'image'.


<div class="alert alert-info">
Cela semble assez cohérent. Le modèle semble avoir réussi à capter les similarités entre les mots.
</div>

- Préparation des documents (tokenization)

In [10]:
maxlen = int(average_length)  # on va prendre ici la taille moyenne (en nombre de mot)
                              # des documents de notre corpus, calculée précédemment,
                              # que j'arrondis en entier pour éviter les erreurs

# On réutilise documents (défini lors de la création et entraînement du modèle Word2Vec),
# documents contient data['corpus']
print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents)
x_documents = pad_sequences(tokenizer.texts_to_sequences(documents),
                            maxlen=maxlen,  # On ne prend que les maxlen premiers mots de chaque document
                            padding='post'  # Si le document est plus petit que maxlen, on complète avec des 0
) 
                                                   
num_words = len(tokenizer.word_index) + 1  # + 1 : on ajoute 1 pour inclure un index supplémentaire.
                                           # Cet index supplémentaire est ajouté pour tenir compte des mots inconnus
                                           # qui n'apparaissent pas dans le jeu de données d'entraînement initial.
                                           
print("Nombre de mots uniques : %i" % num_words)
print("Premier document encodé :")
print(x_documents[0])

Fit Tokenizer ...
Nombre de mots uniques : 3708
Premier document encodé :
[ 290 1561    2  935   13  222  290   27   11  245   36   39  161   40
  245    2   36 1627  245  167    3  245   66   27  313  463]


- Création de la matrice d'embedding

In [11]:
print("Création de la matrice d'embedding...")

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # + 1 pour les même raisons que précédemment.
embedding_matrix = np.zeros((vocab_size, w2v_size))  # w2v_size a été déterminé à la création et
                                                     # entraînement du modèle Word2Vec, pour représenter
                                                     # la taille max des vecteurs. La matrice qu'on crée ici
                                                     # doit avoir la même "largeur".

i=0
j=0   
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)  # Taux d'intégration de mots : Ce taux indique la proportion de mots du corpus
                             # pour lesquels un vecteur d'embedding a été trouvé dans le modèle Word2Vec.
                             # Un taux élevé suggère que la plupart des mots du corpus ont été trouvés dans
                             # le modèle Word2Vec, ce qui signifie que la couverture du modèle d'embedding
                             # est suffisamment large pour représenter le vocabulaire du corpus. 

print("Taux d'intégration de mots :", word_rate)
print(f"Dimensions de la matrice d'embedding : {embedding_matrix.shape}")

Création de la matrice d'embedding...
Taux d'intégration de mots : 1.0
Dimensions de la matrice d'embedding : (3708, 100)


Le taux d'intégration de mots est bon, la couverture du modèle d'embedding est suffisamment large pour représenter le vocabulaire du corpus.

- Création du modèle d'embedding

In [12]:
# Création du modèle

input = Input(shape=(len(x_documents), maxlen),  # On réutilise le même maxlen
                                                 # que celui défini lors de la préparation
                                                 # des documents (tokenization)
            dtype='float64'
)

word_input = Input(shape=(maxlen,),dtype='float64')

word_embedding = Embedding(input_dim=vocab_size,
                           output_dim=w2v_size,
                           weights = [embedding_matrix],
                           input_length=maxlen)(word_input)

word_vec=GlobalAveragePooling1D()(word_embedding)

embed_model = Model([word_input],word_vec)

embed_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 26)]              0         
                                                                 
 embedding (Embedding)       (None, 26, 100)           370800    
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
Total params: 370,800
Trainable params: 370,800
Non-trainable params: 0
_________________________________________________________________


- Exécution du modèle

In [13]:
word2vec_embedded = embed_model.predict(x_documents, verbose=0)
print(f"Dimensions de 'word2vec_embedded' : {word2vec_embedded.shape}")
print("Premier document encodé :")
print(word2vec_embedded[0])

Dimensions de 'word2vec_embedded' : (46498, 100)
Premier document encodé :
[-0.40545294 -0.09687266  0.19511618 -0.00361695  0.01519568 -0.23669513
  0.9939283   0.60402423 -0.01854311  0.39658058  0.71295476  0.32810813
 -0.83919275 -0.41721785 -0.6430797   0.16601823  0.16116169  0.88745284
  0.14624248  0.05588098  1.0842241   0.3058086  -0.35636872 -0.7434487
  0.0683717   0.28726614  0.44141242 -0.73166156 -0.05949076 -0.6878446
  0.5763946  -0.97624964  0.27273828  0.3472854  -0.18710653 -0.18883404
 -0.7774018  -0.68133426 -0.02940808  0.00290322 -0.19713283  0.11295965
 -1.1706913  -0.32782844  0.33893853  0.8681342  -0.31117445  0.07136986
 -0.44696397 -0.67729694  0.33093786  0.38690192  1.1631516   0.8139627
  0.50896585 -0.28680724  0.7326416   0.11956118  0.28780574  0.11909994
  0.504765    0.2388108  -0.56621665  0.520311    0.07890415 -0.0025532
  0.3769908   0.16106261 -0.3455995  -0.21254331  0.5828712  -0.33095977
  0.0181898   0.38551027  0.61834896  0.0210034   0.6